In [1]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.ml.functions import vector_to_array
from pyspark.sql import functions as F
import argparse

/opt/conda/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
table = 'reddit_table'
model_path = '/user/bda_reddit_pw/models/sentiment_model'
result_path = '/user/bda_reddit_pw/models/results/sentiment_results'
day_shift = 20

spark = SparkSession.builder \
    .master('local[1]') \
    .appName('Sentiment Prediction') \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

In [3]:
loaded_model = PipelineModel.load(model_path)

In [4]:
test_df = spark.sql(f'SELECT * FROM {table}')

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


In [5]:
test_df = test_df.withColumnRenamed('selftext', 'text')

In [6]:
test_df.show()

+--------------------+------------+--------------+----+-----+---+----+
|                text| created_utc|cryptocurrency|year|month|day|hour|
+--------------------+------------+--------------+----+-----+---+----+
|                    |1.66458368E9|        solana|2022|   10|  1|   0|
|disclaimer long b...|1.66458483E9|       cardano|2022|   10|  1|   0|
|bob send address ...|1.66458573E9|       bitcoin|2022|   10|  1|   0|
|back get much res...|1.66458944E9|      dogecoin|2022|   10|  1|   1|
|glaring lack lite...| 1.6645897E9|       bitcoin|2022|   10|  1|   2|
|sitting wallet bl...| 1.6645952E9|       bitcoin|2022|   10|  1|   3|
|huge advocate thi...|1.66459558E9|      ethereum|2022|   10|  1|   3|
|know time chain w...|1.66459661E9|       bitcoin|2022|   10|  1|   3|
|discord invite li...|1.66459686E9|           xrp|2022|   10|  1|   4|
|      safe left know|1.66459994E9|       bitcoin|2022|   10|  1|   4|
|please utilize st...|1.66460083E9|       bitcoin|2022|   10|  1|   5|
|see l

In [7]:
prediction = loaded_model.transform(test_df)

In [8]:
results = (prediction.withColumn("xs", vector_to_array("probability"))).select(['prediction'] + [F.col("xs")[1]] + ['created_utc'])
results = results.withColumnRenamed('xs[1]', 'probability')
results = results.withColumn('created_utc', F.date_format(results.created_utc.cast('timestamp'), "yyyy-MM-dd HH:00:00"))
results = results.groupBy('created_utc').agg(F.avg('probability'), F.avg('prediction'))

In [9]:
results = results.withColumnRenamed('avg(probability)', 'avg_probability')
results = results.withColumnRenamed('avg(prediction)', 'avg_prediction')
results = results.withColumnRenamed('created_utc', 'date')
results = results.withColumn('date', F.date_sub(results['date'], day_shift))
results.show()

+----------+------------------+------------------+
|      date|   avg_probability|    avg_prediction|
+----------+------------------+------------------+
|2022-09-16|0.9999999999999988|               1.0|
|2022-09-15|0.5311723408744236|               0.5|
|2022-09-15|0.9145162730257073|               1.0|
|2022-09-15|0.3102653971279222|0.3333333333333333|
|2022-09-14|0.7993924780262095|               0.8|
|2022-09-17|0.9412943891486791|               1.0|
|2022-09-15|0.9998260500941284|               1.0|
|2022-09-17|0.5082690101268774|               0.5|
|2022-09-16|0.6666448334045235|0.6666666666666666|
|2022-09-15|0.8561052848048938|              0.75|
|2022-09-15|0.9992252032710601|               1.0|
|2022-09-16|0.5000129575610124|               0.5|
|2022-09-17|0.5800103837403789|0.5714285714285714|
|2022-09-15|0.4974571990467472|               0.5|
|2022-09-16|0.9999741274133681|               1.0|
|2022-09-17|0.9926435053908795|               1.0|
|2022-09-16|0.9996505554303563|

In [10]:
crypto_df = spark.sql(f'SELECT * FROM crypto_table')

In [11]:
crypto_df = crypto_df.withColumn('date', F.date_format(crypto_df.date.cast('timestamp'), "yyyy-MM-dd HH:00:00"))

In [12]:
results.join(crypto_df,results.date ==  crypto_df.date,"left").show()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 48214)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46273)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11737/4115056868.py", line 1, in <module>
    results.join(crypto_df,results.date ==  crypto_df.date,"left").show()
  File "/usr/lib/spark/python/pyspark/sql/dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o268.showString



ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46273)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11737/4115056868.py", line 1, in <module>
    results.join(crypto_df,results.date ==  crypto_df.date,"left").show()
  File "/usr/lib/spark/python/pyspark/sql/dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o268.showString



ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46273)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11737/4115056868.py", line 1, in <module>
    results.join(crypto_df,results.date ==  crypto_df.date,"left").show()
  File "/usr/lib/spark/python/pyspark/sql/dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o268.showString



ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46273)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11737/4115056868.py", line 1, in <module>
    results.join(crypto_df,results.date ==  crypto_df.date,"left").show()
  File "/usr/lib/spark/python/pyspark/sql/dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o268.showString



...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46273)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11737/4115056868.py", line 1, in <module>
    results.join(crypto_df,results.date ==  crypto_df.date,"left").show()
  File "/usr/lib/spark/python/pyspark/sql/dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o268.showString



ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46273)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11737/4115056868.py", line 1, in <module>
    results.join(crypto_df,results.date ==  crypto_df.date,"left").show()
  File "/usr/lib/spark/python/pyspark/sql/dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o268.showString



ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46273)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11737/4115056868.py", line 1, in <module>
    results.join(crypto_df,results.date ==  crypto_df.date,"left").show()
  File "/usr/lib/spark/python/pyspark/sql/dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o268.showString



ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46273)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11737/4115056868.py", line 1, in <module>
    results.join(crypto_df,results.date ==  crypto_df.date,"left").show()
  File "/usr/lib/spark/python/pyspark/sql/dataframe.py", line 484, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(
py4j.protocol.Py4JError: An error occurred while calling o268.showString



Py4JError: An error occurred while calling o268.showString

In [ ]:
# cmd = f'hdfs dfs -rmr {results_path}'
# os.system(cmd)
# test_df.write.csv(results, header=True)